In [7]:
import pandas as pd
import sqlite3

name = 'Alejandra Camino'

conn = sqlite3.connect("normalized.db")
sql_statement = """ 
        
        WITH sales_quarter AS (
    SELECT
        CASE 
            WHEN 0 + strftime('%m', o.OrderDate) BETWEEN 1 AND 3 THEN 'Q1'
            WHEN 0 + strftime('%m', o.OrderDate) BETWEEN 4 AND 6 THEN 'Q2'
            WHEN 0 + strftime('%m', o.OrderDate) BETWEEN 7 AND 9 THEN 'Q3'
            WHEN 0 + strftime('%m', o.OrderDate) BETWEEN 10 AND 12 THEN 'Q4'
            END AS Quarter,
            CAST(strftime('%Y', o.OrderDate) AS INT) Year, c.CustomerID,
            ROUND(SUM(p.ProductUnitPrice * o.QuantityOrdered)) AS Total
            FROM OrderDetail o
            JOIN Customer c ON c.CustomerID = o.CustomerID
            JOIN Product p ON p.ProductID = o.ProductID
            GROUP BY c.CustomerID, Year, Quarter
            ORDER BY Year, Total DESC 
    )
    SELECT * FROM ( 
    SELECT Quarter, Year, CustomerID, Total, 
    RANK() OVER(PARTITION BY Year,Quarter ORDER BY Year,Quarter ASC) CustomerRank FROM sales_quarter
    ) 
    WHERE CustomerRank BETWEEN 1 AND 5

    

     """
df = pd.read_sql_query(sql_statement, conn)
display(df)
conn.close()


,Quarter,Year,CustomerID,Total,CustomerRank
0,Q3,2012,79,561433.0,1
1,Q3,2012,29,536849.0,1
2,Q3,2012,8,502864.0,1
3,Q3,2012,30,500478.0,1
4,Q3,2012,71,483266.0,1
...,...,...,...,...,...
1360,Q1,2016,64,57697.0,1
1361,Q1,2016,44,53756.0,1
1362,Q1,2016,79,52039.0,1
1363,Q1,2016,60,46076.0,1


In [8]:
conn.close()